In [ ]:
import pandas as pd
import json
from collections import Counter
import math
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import matplotlib.transforms as mtrans
import matplotlib
%matplotlib inline

path = "AllFrequent.csv"


df = pd.read_csv("AllFrequent.csv")
df = df[['Count', 'Developers']]
df['Instances'] = df.Count
df['Devs'] = df.Developers
df = df.drop('Count', axis=1)
df = df.drop('Developers', axis=1)
df.head()

In [ ]:
groups = [
    (2, 2),
    (3, 3),
    (4, 4),
    (5, 5),
    (6, 6),
    (7, 7),
    (8, 8),
    (9, 9),
    (10, 15),
    (16, 25),
    (26, 50),
    (50, 100),
    (122, 122)
]

def group_to_label(g):
    if g[0] == g[1]:
        return str(g[0])
    else:
        return '{0:03}'.format(g[0]) + '-' + '{0:03}'.format(g[1])

def get_group(row_index):
    for g in groups:
        instances = df.iloc[row_index].Instances.item()
        if g[0] <= instances and instances <= g[1]:
            return group_to_label(g), df.iloc[row_index].Devs
    
freq_data = df.groupby(get_group).size()
#freq_data = freq_data.reindex(index=freq_data.index[::-1])
freq_data.index = pd.MultiIndex.from_tuples(freq_data.index, names=['Instances', 'Devs'])
freq_data = freq_data.unstack(fill_value=0)
freq_data = freq_data.reindex(index=list(map(group_to_label, groups))[::-1])

In [ ]:
matplotlib.rc('xtick', labelsize=40) 
matplotlib.rc('ytick', labelsize=40) 
sns.set(font_scale=1.5)

fig, ax = plt.subplots(ncols=1, nrows=1, sharex=True)
fig.set_size_inches(12, 12)

mask = freq_data.applymap(lambda x: True if x == 0 else False)

ax = sns.heatmap(freq_data,
    vmin = 0,
    vmax = 50,
    cmap='rocket_r',
    linewidths=0.5,
    cbar = True,
    mask = mask
)

ax.xaxis.label.set_text("# Developers")
ax.xaxis.label.set_fontsize(40)
plt.xticks(fontsize=25)

# ax.tick_params(axis='y', labelrotation=0)
ax.yaxis.label.set_text("# Instances")
ax.yaxis.label.set_fontsize(40)
plt.yticks(fontsize=25)


line = plt.Line2D(
    [0.1, 0.75],
    [0.59,0.59],
    transform=fig.transFigure,
    color="black",
    dashes=(4,4),
    linewidth=1
)
fig.add_artist(line)

# plt.axhspan(10, 13, facecolor='0.4', alpha=0.5)
# plt.axvspan(0, 2, facecolor='0.4', alpha=0.5)

hatch_args = {
    'edgecolor': '0.4',
    'facecolor': 'none',
    'alpha'    : 0.5,
    'hatch'    : 'x'
}

grey_args = {
    'facecolor': '0.4',
    'alpha'    : 0.5,
}
style_args = hatch_args
# style_args = grey_args

plt.axhspan(10, 13, **style_args)
rect = matplotlib.patches.Rectangle((0, 0), 2, 10, **style_args)

ax.add_patch(rect)

plt.savefig('plot.pdf', bbox_inches='tight')